In [ ]:
# load package
import matplotlib.pyplot as plt
import torch
import numpy as np
import einops

from .src.visionts import VisionTS
from monash import gluonts_util

In [ ]:
# load args and pretrain model

data_name = 'electricity'
ARCH = 'mae_base' # choose from {'mae_base', 'mae_large', 'mae_huge'}. We recommend 'mae_base'
DEVICE = 'cuda:3'
# The following code will automatically download MAE (base) checkpoint file into ./ckpt/ if not exists.
model = VisionTS(ARCH, ckpt_dir='./ckpt/').to(DEVICE)

In [1]:
# load data


In [2]:
# show as time serie and image

imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])

def show_image(image, title=''):
    # image is [H, W, 3]
    assert image.shape[2] == 3
    plt.imshow(torch.clip((image.cpu() * imagenet_std + imagenet_mean) * 255, 0, 255).int())
    plt.title(title, fontsize=16)
    plt.axis('off')

In [ ]:
# Compare the prediction with the ground truth